# Kaggle Dataset Download

In [1]:
! pip install kaggle

In [9]:
! mkdir ".kaggle"

A subdirectory or file .kaggle already exists.


In [20]:
! kaggle competitions download -c feedback-prize-2021


  0%|          | 0.00/32.4M [00:00<?, ?B/s]
  3%|3         | 1.00M/32.4M [00:00<00:05, 5.83MB/s]
 12%|#2        | 4.00M/32.4M [00:00<00:01, 15.8MB/s]
 19%|#8        | 6.00M/32.4M [00:00<00:01, 16.1MB/s]
 25%|##4       | 8.00M/32.4M [00:00<00:01, 15.7MB/s]
 31%|###       | 10.0M/32.4M [00:00<00:01, 16.2MB/s]
 37%|###7      | 12.0M/32.4M [00:00<00:01, 14.8MB/s]
 43%|####3     | 14.0M/32.4M [00:00<00:01, 14.8MB/s]
 49%|####9     | 16.0M/32.4M [00:01<00:01, 15.1MB/s]
 56%|#####5    | 18.0M/32.4M [00:01<00:00, 15.4MB/s]
 62%|######1   | 20.0M/32.4M [00:01<00:00, 15.2MB/s]
 68%|######7   | 22.0M/32.4M [00:01<00:00, 15.0MB/s]
 74%|#######4  | 24.0M/32.4M [00:01<00:00, 14.9MB/s]
 80%|########  | 26.0M/32.4M [00:01<00:00, 14.9MB/s]
 86%|########6 | 28.0M/32.4M [00:01<00:00, 14.8MB/s]
 93%|#########2| 30.0M/32.4M [00:02<00:00, 15.3MB/s]
 99%|#########8| 32.0M/32.4M [00:02<00:00, 16.3MB/s]
100%|##########| 32.4M/32.4M [00:02<00:00, 15.2MB/s]


In [21]:
cd dataset

C:\Users\pc\PycharmProjects\AI Projects\NLP\dataset


In [25]:
import zipfile
with zipfile.ZipFile("feedback-prize-2021.zip", "r") as zip_ref:
    zip_ref.extractall("dataset")

# Importing Libraries

In [33]:
! pip install spacy

  Using cached langcodes-3.3.0-py3-none-any.whl (181 kB)
  Using cached spacy_legacy-3.0.9-py2.py3-none-any.whl (20 kB)
  Using cached smart_open-5.2.1-py3-none-any.whl (58 kB)


In [36]:
import pandas as pd
import numpy as np
from glob import glob

import matplotlib.pyplot as plt 
import matplotlib.style as style
style.use('fivethirtyeight')
from matplotlib.ticker import FuncFormatter

from nltk.corpus import stopwords
from tqdm.notebook import tqdm

import warnings
warnings.filterwarnings('ignore')
import spacy
import os

from sklearn.feature_extraction.text import CountVectorizer

In [44]:
df_train = pd.read_csv("./dataset/train.csv")
df_train.head(10)

,id,discourse_id,discourse_start,discourse_end,discourse_text,discourse_type,discourse_type_num,predictionstring
0,423A1CA112E2,1.622628e+12,8.0,229.0,Modern humans today are always on their phone....,Lead,Lead 1,1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 1...
1,423A1CA112E2,1.622628e+12,230.0,312.0,They are some really bad consequences when stu...,Position,Position 1,45 46 47 48 49 50 51 52 53 54 55 56 57 58 59
2,423A1CA112E2,1.622628e+12,313.0,401.0,Some certain areas in the United States ban ph...,Evidence,Evidence 1,60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75
3,423A1CA112E2,1.622628e+12,402.0,758.0,"When people have phones, they know about certa...",Evidence,Evidence 2,76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 9...
4,423A1CA112E2,1.622628e+12,759.0,886.0,Driving is one of the way how to get around. P...,Claim,Claim 1,139 140 141 142 143 144 145 146 147 148 149 15...
5,423A1CA112E2,1.622628e+12,887.0,1150.0,That's why there's a thing that's called no te...,Evidence,Evidence 3,163 164 165 166 167 168 169 170 171 172 173 17...
6,423A1CA112E2,1.622628e+12,1151.0,1533.0,Sometimes on the news there is either an accid...,Evidence,Evidence 4,211 212 213 214 215 216 217 218 219 220 221 22...
7,423A1CA112E2,1.622628e+12,1534.0,1602.0,Phones are fine to use and it's also the best ...,Claim,Claim 2,282 283 284 285 286 287 288 289 290 291 292 29...
8,423A1CA112E2,1.622628e+12,1603.0,1890.0,If you go through a problem and you can't find...,Evidence,Evidence 5,297 298 299 300 301 302 303 304 305 306 307 30...
9,423A1CA112E2,1.622628e+12,1891.0,2027.0,The news always updated when people do somethi...,Concluding Statement,Concluding Statement 1,355 356 357 358 359 360 361 362 363 364 365 36...


In [46]:
df_train[['discourse_id', 'discourse_start', 'discourse_end']] = df_train[['discourse_id', 'discourse_start', 'discourse_end']].astype(int)

In [48]:
train_set = glob("./dataset/train/*.txt")
test_set = glob("./dataset/test/*.txt")

In [56]:
train_set["discourse_len"] = train_set["discourse_text"].apply(lambda x: len(x.split()))
train_set["pred_len"] = train_set["predictionstring"].apply(lambda x: len(x.split()))

cols = ['discourse_id', 'discourse_text', 'discourse_type', 'predictionstring', 'disocurse_len', 'pred_len']

TypeError: string indices must be integers